Imports

In [30]:
from PIL import Image
import numpy as np
import math
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [44]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

In [46]:
print(list(convertToBinary(256)))

['1', '0', '0', '0', '0', '0', '0', '0', '0']


- Binary to Decimal Converter

In [32]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [33]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


## 1D Chaotic Map

- Logistic Map

In [34]:
def LogisticMap(dimension, key, k):
    
    x = key
    a = 3.6

    bitSequence = []
    matrix = []

    for i in range(dimension * k):
        x_next = a * x * (1 - x) 
        x = x_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % k == k - 1:
            decimal = binaryToDecimal(bitSequence)
            matrix.append(decimal)
            bitSequence = []

    return matrix

## 2D Chaotic Map

- Hennon Map Implementation

In [35]:
def HennonMap(dimension, key, k):

    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = dimension * dimension * k 
    bitSequenceSize = k
    byteArraySize = dimension * k
    # Each bitSequence contains k bits
    bitSequence = []    
    # Each byteArray contains m bitSequence
    byteArray = []      
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            decimal = binaryToDecimal(bitSequence)
            byteArray.append(decimal)
            bitSequence = []
        
        # If Byte Sequence has k bytes, add it to Matrix.
        if i % byteArraySize == byteArraySize - 1:
            matrix.append(byteArray)
            byteArray = []

    return matrix

In [36]:
h = HennonMap(4, [0.1,0.1], 3)
print(h)

[[5, 2, 1, 6], [4, 2, 4, 7], [2, 1, 2, 3], [7, 7, 5, 2]]


- Hennon Map (Binary)

In [37]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

In [38]:
h = HennonMapBinary([3,4], [0.1,0.2])
print(h)

[[1, 0, 0, 1], [0, 1, 0, 1], [0, 1, 1, 1]]


- Hennon Map (Indexes Only)

In [39]:
def HennonMapNew(required, key, k):

    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    chosen_pixels = {}
    chosen_pixels_count = 0

    bitSequenceSize = k
    # Each bitSequence contains 8 bits
    bitSequence = []
    indices = []

    while True:
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = "0"
        else:
            bit = "1"

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Indices.
        if len(bitSequence) == bitSequenceSize:
            indices.append("".join(bitSequence))
            bitSequence = []
        
        # If Indices Sequence has 2 values, add it to chosen pixels.
        if len(indices) == 2:
            if (indices[0],indices[1]) not in chosen_pixels:
                chosen_pixels[(indices[0],indices[1])] = 1
                chosen_pixels_count += 1
                if chosen_pixels_count == required:
                    return chosen_pixels
            indices = []
            
    return -1

In [40]:
print(HennonMapNew(5,[0.1,0.1],3))

{('101', '010'): 1, ('001', '110'): 1, ('100', '010'): 1, ('100', '111'): 1, ('010', '001'): 1}


## Encoding


In [55]:
def encode(imageName, filename, key, poly):

    # read the image
    imagehandler = Image.open(imageName) 
    image = imagehandler.load()
    image_size = imagehandler.size

    # Get secret data
    filehandler = open(filename,"r")
    secret_data = filehandler.readline()

    # convert data to binary
    binary_secret_data = convertToBinary(secret_data)
    data_len = len(binary_secret_data)

    print(data_len)

    # Build chaotic map and derive some available pixels
    map = HennonMapBinary(image_size, key)

    availablePixels = []

    for i in range(image_size[0]):
        for j in range(0,image_size[1]):
            availablePixels.append([i,j,0])

    # check if encoding is possible
    if data_len > len(availablePixels)*6:
        raise ValueError("[!] Insufficient space. Please change key or data.")

    # Build LFSR
    lfsr = LFSR(initstate = [1, 1, 1], fpoly=poly, counter_start_zero=True)

    index = 0

    for i in range(0, data_len, 2):

        # Choose an unused pixel using LFSR from Chaotic Map

        index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
        lfsr.next()

        count = 0
        while availablePixels[index][2] >= 2:
            index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
            print(index)
            lfsr.next()
            count += 1
            if count > 100:
                print("exceeded")
                return

        '''
        EACH PIXEL CAN BE USED AT MAX 3 TIMES. WE KEEP COUNT OF NUMBER OF USES AS USAGESTATE.

        IF USAGESTATE IS 0: XOR 2 DATA BITS WITH 2 LSB OF RED VALUE
        IF USAGESTATE IS 1: XOR 2 DATA BITS WITH 2 LSB OF BLUE VALUE
        IF USAGESTATE IS 2: XOR 2 DATA BITS WITH 2 LSB OF GREEN VALUE
        '''

        row, col, usageState = availablePixels[index][0], availablePixels[index][1], availablePixels[index][2]
        
        # Update with Red Value
        if usageState == 0:
            red = list(convertToBinary(image[row,col][0]))
            red[-2] = str(int(red[-2]) ^ int(binary_secret_data[i]))
            red[-1] = str(int(red[-1]) ^ int(binary_secret_data[i+1]))
            red = "".join(red)
            red = binaryToDecimal(red)
            image[row,col] = (red, image[row,col][1], image[row,col][2])

        # Update with Blue Value
        elif usageState == 1:
            blue = list(convertToBinary(image[row,col][1]))
            blue[-2] = str(int(blue[-2]) ^ int(binary_secret_data[i]))
            blue[-1] = str(int(blue[-1]) ^ int(binary_secret_data[i+1]))
            blue = "".join(blue)
            blue = binaryToDecimal(blue)
            image[row,col] = (image[row,col][0], blue, image[row,col][2])
        
        # Update with Green Value
        else:
            green = list(convertToBinary(image[row,col][0]))
            green[-2] = str(int(green[-2]) ^ int(binary_secret_data[i]))
            green[-1] = str(int(green[-1]) ^ int(binary_secret_data[i+1]))
            green = "".join(green)
            green = binaryToDecimal(green)
            image[row,col] = (image[row,col][0], image[row,col][1], green)
        
        availablePixels[index][2] += 1

    # Save the final image.
    imagehandler.save("images\encoded_image.png", "PNG")

In [57]:
encode("images\original_image.jpeg", "text\input.txt", [0.1, 0.1], [3,2])

216


In [58]:
print(getImageMatrix("images\original_image.jpeg")[0][0])
print(getImageMatrix("images\encoded_image.png")[0][0])

[(211, 210, 205), (211, 210, 205), (211, 210, 205), (211, 210, 205), (211, 210, 205), (210, 209, 204), (210, 209, 204), (210, 209, 204), (210, 209, 204), (210, 209, 204), (208, 209, 203), (208, 209, 203), (208, 209, 203), (208, 209, 203), (208, 209, 203), (208, 209, 203), (208, 209, 204), (208, 209, 204), (208, 209, 204), (208, 209, 204), (208, 209, 204), (208, 209, 204), (208, 209, 204), (208, 209, 204), (207, 208, 203), (207, 208, 203), (206, 208, 203), (207, 209, 204), (207, 209, 204), (207, 209, 204), (207, 209, 204), (207, 209, 204), (208, 209, 204), (209, 210, 205), (209, 210, 205), (209, 210, 205), (209, 210, 205), (209, 210, 205), (209, 210, 205), (210, 211, 206), (210, 211, 205), (210, 211, 205), (210, 211, 205), (210, 211, 205), (211, 212, 206), (211, 212, 206), (211, 212, 206), (211, 212, 206), (213, 212, 207), (213, 212, 207), (213, 212, 207), (213, 212, 207), (213, 212, 207), (214, 213, 208), (214, 213, 208), (214, 213, 208), (214, 213, 208), (214, 213, 208), (215, 214, 20